In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import roc_auc_score
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import Trials

from xgboost import *

try:
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
except:
    !pip install --upgrade shap-hypetune
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA

import warnings
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
import importlib
import s3finance as s3c
from labellines import labelLines
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from functools import partialmethod
from category_encoders import TargetEncoder
import seaborn as sns
from matplotlib import pyplot as plt, dates
from sklearn.metrics import confusion_matrix, auc, roc_curve
from statistics import mean
import pandas_market_calendars as mcal
import joblib
import time

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
def binary_performances(y_true, y_prob, thresh=0.5, labels=['Positives','Negatives']):
    
    shape = y_prob.shape
    if len(shape) > 1:
        if shape[1] > 2:
            raise ValueError('A binary class problem is required')
        else:
            y_prob = y_prob[:,1]
    
    plt.figure(figsize=[15,4])
    
    #1 -- Confusion matrix
    cm = confusion_matrix(y_true, (y_prob>thresh).astype(int))
    
    plt.subplot(131)
    ax = sns.heatmap(cm, annot=True, cmap='Blues', cbar=False, 
                     annot_kws={"size": 14}, fmt='g')
    cmlabels = ['True Negatives', 'False Positives',
               'False Negatives', 'True Positives']
    for i,t in enumerate(ax.texts):
        t.set_text(t.get_text() + "\n" + cmlabels[i])
    plt.title('Confusion Matrix', size=15)
    plt.xlabel('Predicted Values', size=13)
    plt.ylabel('True Values', size=13)
      
    #2 -- Distributions of Predicted Probabilities of both classes
    plt.subplot(132)
    plt.hist(y_prob[y_true==1], density=True, bins=25,
             alpha=.5, color='green',  label=labels[0])
    plt.hist(y_prob[y_true==0], density=True, bins=25,
             alpha=.5, color='red', label=labels[1])
    plt.axvline(thresh, color='blue', linestyle='--', label='Boundary')
    plt.xlim([0,1])
    plt.title('Distributions of Predictions', size=15)
    plt.xlabel('Positive Probability (predicted)', size=13)
    plt.ylabel('Samples (normalized scale)', size=13)
    plt.legend(loc="upper right")
    
    #3 -- ROC curve with annotated decision point
    fp_rates, tp_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fp_rates, tp_rates)
    plt.subplot(133)
    plt.plot(fp_rates, tp_rates, color='orange',
             lw=1, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], lw=1, linestyle='--', color='grey')
    tn, fp, fn, tp = [i for i in cm.ravel()]
    plt.plot(fp/(fp+tn), tp/(tp+fn), 'bo', markersize=8, label='Decision Point')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', size=13)
    plt.ylabel('True Positive Rate', size=13)
    plt.title('ROC Curve', size=15)
    plt.legend(loc="lower right")
    plt.subplots_adjust(wspace=.3)
    plt.show()

    tn, fp, fn, tp = [i for i in cm.ravel()]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F1 = 2*(precision * recall) / (precision + recall)
    results = {
        "Precision": precision, "Recall": recall,
        "F1 Score": F1, "AUC": roc_auc
    }
    
    prints = [f"{kpi}: {round(score, 3)}" for kpi,score in results.items()]
    prints = ' | '.join(prints)
    print(prints)
    
    return results

def AUC(y_true, y_hat):
    return 'auc', roc_auc_score(y_true, y_hat), True

In [ ]:
# set up access to access class
#
AWS_BUCKET = "w210-wrds-data"
importlib.reload(s3c)
#s3f = s3c.s3finance(AWS_BUCKET)
version='1658902110.776109'
s3f = s3c.s3finance(AWS_BUCKET,version=version)

In [ ]:
#
# Read all Price Data
#
# get all the financial ratios
all_ratios = s3f.getsummaryallratiopricesfinancial()

In [ ]:
# get all the stock prices
all_prices = s3f.getsummaryprices()
#all_prices = None

In [ ]:
# get SPY prices
#s3f = s3c.s3finance(AWS_BUCKET,version='1658134425.075057')
spyprices=s3f.getstockprices("SPY")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1,0.2],
    'num_leaves': [25, 35,100],
    'n_estimators': [100,500,1000,2000],
    'max_depth':  [3, 5, 6, 10, 15, 20],
    'colsample_bytree': [0.3, 0.7, 1.0]
}

params_random = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

param_dist = {
    'learning_rate': stats.uniform(0.01, 0.25),
    'colsample_bytree': np.arange(0.4, 1.0, 0.1),
    'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
#    'num_leaves': stats.randint(20,100),
    'n_estimators': [100,500,1000,2000],
    'max_depth': [3, 5, 6, 10]
}

param_dist_hyperopt = {
    'max_depth': 15 + hp.randint('num_leaves', 5), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}



In [ ]:
#s3f = s3c.s3finance(AWS_BUCKET)

In [ ]:
# predict for individual stock
#
import time

def runcyclesticker(all_ratios,ncycles=12,period=10,step=1, years_ahead = 1, predict_start=None,train_start='2000-01-01',end_period='2021-01-01', ticker=[], 
                    c1=None, min_price=5, market_cap=10000,output_analysis=True,model_data_filename="model-period-{period}-year-{cycles}-cycles", savemodel_cycle=12):
    
    if predict_start is None:
        predict_start = (pd.to_datetime(train_start) + relativedelta(years=(period+years_ahead))).strftime("%Y-%m-%d")  
 
    #print(predict_start)
    start_date_predict = predict_start
    start_date_train = train_start
    end_date = end_period
    train_period = period # number of years
    #+ relativedelta(years=1)
    
    tpredict = np.arange(start_date_predict, end_date,np.timedelta64(step, 'M'),  dtype='datetime64[M]').astype('datetime64[D]').tolist() 
    ttrain = np.arange(start_date_train, end_date,np.timedelta64(step, 'M'),  dtype='datetime64[M]').astype('datetime64[D]').tolist()
    listcycles = list(zip(range(ncycles),tpredict,ttrain))

    results = pd.DataFrame(data=[],columns=['date'] + c1 + ['score'],index = list(range(len(listcycles))))
    shaplist = []
    models = []
    for i, p,t in tqdm(listcycles,disable=True):
        start_tm = time.time()
        train_set = all_ratios[(all_ratios['monthly_date'] >= t) & 
                               (all_ratios['monthly_date'] < (t+ relativedelta(years=train_period))) &
                               (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]
        encoder = TargetEncoder()
        train_set['stock_encoder'] = encoder.fit_transform(train_set['cusip'].astype(str), train_set['outperformed'])
        if len(ticker)> 0:
            test_set=all_ratios[(all_ratios['monthly_date'] >= p) & (all_ratios['monthly_date'] < (p+ relativedelta(months=step))) & 
                                   (all_ratios['ticker'].isin(ticker)) & 
                                (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]
        else:
            test_set=all_ratios[(all_ratios['monthly_date'] >= p) & 
                                (all_ratios['monthly_date'] < (p+ relativedelta(months=step))) 
                                & (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]

        encoder = TargetEncoder()
        test_set['stock_encoder'] = encoder.fit_transform(test_set['cusip'].astype(str), test_set['outperformed'])
        X = train_set[c1]
        y = train_set['outperformed']
        Xt = test_set[c1]
        yt = test_set['outperformed']
        print(len(X),len(y),len(Xt),len(yt))
        # clf_xgb2 = XGBClassifier(n_estimators=1000, random_state=0, verbosity=0, n_jobs=-1,learning_rate =0.1,scoring='roc_auc') #n_jobs=-1)
        clf_xgb = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=10,
                                min_child_weight=1, gamma=0.3, subsample=0.8,
                                colsample_bytree=0.8, 
                                objective= 'binary:logistic', nthread=4, 
                                scale_pos_weight=1, seed=27,n_jobs=-1,
                                early_stopping_rounds=6,
                                 #scoring='roc_auc'
                                )
        model = RandomizedSearchCV(estimator=clf_xgb,
                                    param_distributions=param_dist,
                                    scoring='accuracy',
                                    # scoring='f1',
                                    # scoring='roc_auc',
                                    n_iter=50,                                   
                                    verbose=0,n_jobs=-1)

        model.fit(X, y, eval_set=[(Xt, yt)], verbose=0) #  

        # Fits the explainer
        explainer = shap.Explainer(model.predict, Xt,seed=42)
        # Calculates the SHAP values - It takes some time
        shap_values = explainer(Xt)
        shaplist.append(shap_values)
        results.iloc[i] = [p] + list(shap_values[0].values) + [model.score(Xt, yt)]
        if output_analysis:
            binary_performances(yt, model.predict_proba(Xt),thresh=0.5) # thresh = 0.5)
            shap.plots.bar(shap_values,max_display=50)
            shap.summary_plot(shap_values,Xt,max_display=50)
        models.append([model,Xt,yt,test_set])
        if ((i + 1) % savemodel_cycle) == 0:
            # checkpoint models
            s3f.putmodeldata([results,shaplist,models], f"model-period-{period}-year-{i+1}-cycles-{version}.pkl")
        end_tm = time.time()
        print(f'Loop Iteration Time: {(end_tm - start_tm):.2f} seconds, total time: {(len(listcycles)-i)*(end_tm - start_tm):.2f}')
        #print(results)   
    return results,shaplist,models


In [ ]:
keyfields = [x for x in list(all_ratios.select_dtypes(include=['number']).columns) if x not in \
             ['one_yr_chg', 'exchg', 'one_yr_chg_spy', 'ajexdi', 'outperformed','prccd', 'prchd', 'prcld', 'prcod', 'prcstd']]

In [ ]:
%%time
period = 10
cycles = 240
results,shaplist,models = runcyclesticker(all_ratios, ncycles=cycles,period=period,step=1,
                                          train_start='1995-01-01', end_period='2022-08-01',
                                          c1=list(set(keyfields)),
                                          min_price=5, market_cap=300,output_analysis=False)

In [ ]:
s3f.putmodeldata([results,shaplist,models], f"model-period-{period}-year-{cycles}-cycles-{version}-full.pkl")

In [ ]:
resultsf = results.loc[:, results.columns !='date'].astype(float)
results1 = resultsf.loc[:, (resultsf != 0.0).any(axis=0)]
results1['date'] = results7['date'].astype(str)
f = plt.figure()
ax = results1.set_index('date').plot(use_index=True, rot=90,figsize=(30, 20),legend=True,ax=f.gca())
foo=labelLines(ax.get_lines(),  align=False,  color="k")
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()